# Style-Bert-VITS2 (ver 2.6.1) のGoogle Colabでの学習

Google Colab上でStyle-Bert-VITS2の学習を行うことができます。

このnotebookでは、通常使用ではあなたのGoogle Driveにフォルダ`Style-Bert-VITS2`を作り、その内部での作業を行います。他のフォルダには触れません。
Google Driveを使わない場合は、初期設定のところで適切なパスを指定してください。

## 流れ

### 学習を最初からやりたいとき
上から順に実行していけばいいです。音声合成に必要なファイルはGoogle Driveの`Style-Bert-VITS2/model_assets/`に保存されます。また、途中経過も`Style-Bert-VITS2/Data/`に保存されるので、学習を中断したり、途中から再開することもできます。

### 学習を途中から再開したいとき
0と1を行い、3の前処理は飛ばして、4から始めてください。スタイル分け5は、学習が終わったら必要なら行ってください。


## 0. 環境構築

Style-Bert-VITS2の環境をcolab上に構築します。ランタイムがT4等のGPUバックエンドになっていることを確認し、実行してください。

**注意**: このセルを実行した後に「セッションがクラッシュしました」「不明な理由により、セッションがクラッシュしました。」等の警告が出ますが、**無視してそのまま先へ**進んでください。（一度ランタイムを再起動させてnumpy<2を強制させるため `exit()` を呼んでいることからの措置です。）

In [9]:
import os

os.environ["PATH"] += ":/root/.cargo/bin"

!curl -LsSf https://astral.sh/uv/install.sh | sh
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!uv pip install --system -r requirements-colab.txt --no-progress
!python initialize.py --skip_default_models

exit()

downloading uv 0.7.11 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
fatal: destination path 'Style-Bert-VITS2' already exists and is not an empty directory.
/content/Style-Bert-VITS2/Style-Bert-VITS2
Using Python 3.11.13 environment at: /usr
Audited 20 packages in 170ms


In [1]:
# Google driveを使う方はこちらを実行してください。

from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 初期設定

学習とその結果を保存するディレクトリ名を指定します。
Google driveの場合はそのまま実行、カスタマイズしたい方は変更して実行してください。

In [2]:
# 作業ディレクトリを移動
%cd /content/Style-Bert-VITS2/

# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = "/content/drive/MyDrive/Style-Bert-VITS2/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = "/content/drive/MyDrive/Style-Bert-VITS2/model_assets"

import yaml


with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

/content/Style-Bert-VITS2


## 2. 学習に使うデータ準備

すでに音声ファイル（1ファイル2-12秒程度）とその書き起こしデータがある場合は2.2を、ない場合は2.1を実行してください。

### 2.1 音声ファイルからのデータセットの作成（ある人はスキップ可）

音声ファイル（1ファイル2-12秒程度）とその書き起こしのデータセットを持っていない方は、（日本語の）音声ファイルのみから以下の手順でデータセットを作成することができます。Google drive上の`Style-Bert-VITS2/inputs/`フォルダに音声ファイル（wavやmp3等の通常の音声ファイル形式、1ファイルでも複数ファイルでも可）を置いて、下を実行すると、データセットが作られ、自動的に正しい場所へ配置されます。

**2024-06-02のVer 2.5以降**、`inputs/`フォルダにサブフォルダを2個以上作ってそこへ音声ファイルをスタイルに応じて振り分けて置くと、学習の際にサブディレクトリに応じたスタイルが自動的に作成されます。デフォルトスタイルのみでよい場合や手動でスタイルを後で作成する場合は`inputs/`直下へ入れれば大丈夫です。

In [7]:
# 元となる音声ファイル（wav形式）を入れるディレクトリ
input_dir = "/content/drive/MyDrive/Style-Bert-VITS2/inputs/angry"
input_dir = "/content/drive/MyDrive/Style-Bert-VITS2/inputs/natural"

# モデル名（話者名）を入力
model_name = "your_model_name"

# こういうふうに書き起こして欲しいという例文（句読点の入れ方・笑い方や固有名詞等）
initial_prompt = "こんにちは。元気、ですかー？ふふっ、私は……ちゃんと元気だよ！"

!python slice.py -i {input_dir} --model_name {model_name}
!python transcribe.py --model_name {model_name} --initial_prompt {initial_prompt} --use_hf_whisper

06-06 04:52:52 |  INFO  | slice.py:167 | Found 1 audio files.
06-06 04:52:52 |WARNING | slice.py:169 | Output directory /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/raw already exists, deleting...
Using cache found in /root/.cache/torch/hub/litagin02_silero-vad_master
100%|##########| 1/1 [00:00<00:00,  2.09it/s]
06-06 04:52:54 |  INFO  | slice.py:265 | Slice done! Total time: 0.15 min, 1 files.
06-06 04:52:56 |WARNING | transcribe.py:159 | /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/esd.list exists, backing up to /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/esd.list.bak
06-06 04:52:56 |WARNING | transcribe.py:162 | /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/esd.list.bak exists, deleting...
06-06 04:52:56 |  INFO  | transcribe.py:203 | Loading HF Whisper model (openai/whisper-large-v3)
  0%|          | 0/1 [00:00<?, ?it/s]2025-06-06 04:53:00.194333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable t

成功したらそのまま3へ進んでください

### 2.2 音声ファイルと書き起こしデータがすでにある場合

指示に従って適切にデータセットを配置してください。

次のセルを実行して、学習データをいれるフォルダ（1で設定した`dataset_root`）を作成します。

In [ ]:
import os

os.makedirs(dataset_root, exist_ok=True)

まず音声データと、書き起こしテキストを用意してください。

それを次のように配置します。
```
├── Data/
│   ├── {モデルの名前}
│   │   ├── esd.list
│   │   ├── raw/
│   │   │   ├── foo.wav
│   │   │   ├── bar.mp3
│   │   │   ├── style1/
│   │   │   │   ├── baz.wav
│   │   │   │   ├── qux.wav
│   │   │   ├── style2/
│   │   │   │   ├── corge.wav
│   │   │   │   ├── grault.wav
...
```

### 配置の仕方
- 上のように配置すると、`style1/`と`style2/`フォルダの内部（直下以外も含む）に入っている音声ファイルたちから、自動的にデフォルトスタイルに加えて`style1`と`style2`というスタイルが作成されます
- 特にスタイルを作る必要がない場合や、スタイル分類機能等でスタイルを作る場合は、`raw/`フォルダ直下に全てを配置してください。このように`raw/`のサブディレクトリの個数が0または1の場合は、スタイルはデフォルトスタイルのみが作成されます。
- 音声ファイルのフォーマットはwav形式以外にもmp3等の多くの音声ファイルに対応しています

### 書き起こしファイル`esd.list`

`Data/{モデルの名前}/esd.list` ファイルには、以下のフォーマットで各音声ファイルの情報を記述してください。


```
path/to/audio.wav(wavファイル以外でもこう書く)|{話者名}|{言語ID、ZHかJPかEN}|{書き起こしテキスト}
```

- ここで、最初の`path/to/audio.wav`は、`raw/`からの相対パスです。つまり、`raw/foo.wav`の場合は`foo.wav`、`raw/style1/bar.wav`の場合は`style1/bar.wav`となります。
- 拡張子がwavでない場合でも、`esd.list`には`wav`と書いてください、つまり、`raw/bar.mp3`の場合でも`bar.wav`と書いてください。


例：
```
foo.wav|hanako|JP|こんにちは、元気ですか？
bar.wav|taro|JP|はい、聞こえています……。何か用ですか？
style1/baz.wav|hanako|JP|今日はいい天気ですね。
style1/qux.wav|taro|JP|はい、そうですね。
...
english_teacher.wav|Mary|EN|How are you? I'm fine, thank you, and you?
...
```
もちろん日本語話者の単一話者データセットでも構いません。

## 3. 学習の前処理

次に学習の前処理を行います。必要なパラメータをここで指定します。次のセルに設定等を入力して実行してください。「～～かどうか」は`True`もしくは`False`を指定してください。

In [4]:
# 上でつけたフォルダの名前`Data/{model_name}/`
model_name = "your_model_name"

# JP-Extra （日本語特化版）を使うかどうか。日本語の能力が向上する代わりに英語と中国語は使えなくなります。
use_jp_extra = True

# 学習のバッチサイズ。VRAMのはみ出具合に応じて調整してください。
batch_size = 4

# 学習のエポック数（データセットを合計何周するか）。
# 100で多すぎるほどかもしれませんが、もっと多くやると質が上がるのかもしれません。
epochs = 100

# 保存頻度。何ステップごとにモデルを保存するか。分からなければデフォルトのままで。
save_every_steps = 1000

# 音声ファイルの音量を正規化するかどうか
normalize = False

# 音声ファイルの開始・終了にある無音区間を削除するかどうか
trim = False

# 読みのエラーが出た場合にどうするか。
# "raise"ならテキスト前処理が終わったら中断、"skip"なら読めない行は学習に使わない、"use"なら無理やり使う
yomi_error = "skip"

上のセルが実行されたら、次のセルを実行して学習の前処理を行います。

In [5]:
from gradio_tabs.train import preprocess_all
from style_bert_vits2.nlp.japanese import pyopenjtalk_worker


pyopenjtalk_worker.initialize_worker()

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    num_processes=2,
    normalize=normalize,
    trim=trim,
    freeze_EN_bert=False,
    freeze_JP_bert=False,
    freeze_ZH_bert=False,
    freeze_style=False,
    freeze_decoder=False,
    use_jp_extra=use_jp_extra,
    val_per_lang=0,
    log_interval=200,
    yomi_error=yomi_error,
)

KeyboardInterrupt: 

## 4. 学習

前処理が正常に終わったら、学習を行います。次のセルを実行すると学習が始まります。

学習の結果は、上で指定した`save_every_steps`の間隔で、Google Driveの中の`Style-Bert-VITS2/Data/{モデルの名前}/model_assets/`フォルダに保存されます。

このフォルダをダウンロードし、ローカルのStyle-Bert-VITS2の`model_assets`フォルダに上書きすれば、学習結果を使うことができます。

In [8]:
# 上でつけたモデル名を入力。学習を途中からする場合はきちんとモデルが保存されているフォルダ名を入力。
model_name = "your_model_name"


import yaml
from gradio_tabs.train import get_path

paths = get_path(model_name)
dataset_path = str(paths.dataset_path)
config_path = str(paths.config_path)

with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

In [9]:
# 日本語特化版を「使う」場合
!python train_ms_jp_extra.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

2025-06-06 04:54:04.155526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749185644.175936   18584 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749185644.182054   18584 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-06 04:54:04.202489: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
06-06 04:54:11 |  INFO  | train_ms_jp_extra.py:117 | Loading configuration from config 0
06-06 04:54:11 |  INFO  | tr

In [ ]:
# 日本語特化版を「使わない」場合
!python train_ms.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

In [ ]:
# 学習結果を試す・マージ・スタイル分けはこちらから
!python app.py --share

2025-06-06 04:55:07.076621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749185707.105180   18817 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749185707.114034   18817 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-06 04:55:07.145545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
06-06 04:55:11 | DEBUG  | __init__.py:92 | try starting pyopenjtalk worker server
06-06 04:55:12 | DEBUG  | __init__.